# Aegis Scheduled Sync (Strava + TickTick)

This notebook performs a one-shot sync of Strava activities and TickTick tasks into the configured PostgreSQL database.

Environment is read from `.env` at the repo root. It can be executed manually, or scheduled via a job runner (e.g., `cron` using `papermill` / `jupyter nbconvert`).


In [ ]:
# Parameters (can be overridden by environment variables when scheduled)
STRAVA_PER_PAGE = int(os.getenv('STRAVA_PER_PAGE', '200'))
STRAVA_PAGE_START = int(os.getenv('STRAVA_PAGE_START', '1'))
# If set, only fetch Strava activities after this epoch seconds
STRAVA_AFTER_EPOCH = os.getenv('STRAVA_AFTER_EPOCH')
# Optional: comma-separated TickTick project IDs to sync; if empty, sync all
TICKTICK_PROJECT_IDS = os.getenv('TICKTICK_PROJECT_IDS', '').strip()


In [ ]:
# Setup: path, env, and schema
import os, sys, pathlib
from dotenv import load_dotenv

repo_root = pathlib.Path().resolve()
if (repo_root / 'app').exists():
    sys.path.insert(0, str(repo_root))

# Load .env so notebook runs outside FastAPI lifecycle
load_dotenv(override=False)

# Initialize schema (safe if already present) and test DB connection
from app.db import init_schema, db, get_db_url
init_schema()
print('DB URL present:', bool(os.getenv('DATABASE_URL')))
try:
    with db() as conn:
        with conn.cursor() as cur:
            cur.execute('select 1 as ok')
            print('DB connect test:', cur.fetchone()['ok'] == 1)
except Exception as e:
    print('DB connect error:', e)


In [ ]:
# Strava sync
from app.strava import fetch_all_activities, upsert_activities

after_val = None
if STRAVA_AFTER_EPOCH and STRAVA_AFTER_EPOCH.strip():
    try:
        after_val = int(STRAVA_AFTER_EPOCH)
    except Exception:
        print('Warning: invalid STRAVA_AFTER_EPOCH; ignoring')

strava_fetched = []
strava_upserted = 0
try:
    strava_fetched = fetch_all_activities(per_page=STRAVA_PER_PAGE, page_start=STRAVA_PAGE_START, after=after_val)
    strava_upserted = upsert_activities(strava_fetched)
    print({'strava_activities_fetched': len(strava_fetched), 'strava_activities_upserted': strava_upserted})
except Exception as e:
    print('Strava sync error:', e)


In [ ]:
# TickTick sync (all projects or a specified list)
from app.ticktick import list_projects as tt_list_projects, get_project_tasks as tt_get_tasks, upsert_tasks as tt_upsert

ticktick_projects = []
ticktick_tasks_upserted = 0
try:
    if TICKTICK_PROJECT_IDS:
        ids = [p.strip() for p in TICKTICK_PROJECT_IDS.split(',') if p.strip()]
        ticktick_projects = [{'id': pid, 'name': pid} for pid in ids]
    else:
        ticktick_projects = tt_list_projects()
    for p in ticktick_projects:
        pid = p.get('id') if isinstance(p, dict) else None
        if not pid:
            continue
        tasks = tt_get_tasks(pid)
        ticktick_tasks_upserted += tt_upsert(tasks)
    print({'ticktick_projects': len(ticktick_projects), 'ticktick_tasks_upserted': ticktick_tasks_upserted})
except Exception as e:
    print('TickTick sync error:', e)


In [ ]:
# Summary
summary = {
    'strava_activities_fetched': len(strava_fetched) if isinstance(strava_fetched, list) else None,
    'strava_activities_upserted': strava_upserted,
    'ticktick_projects_processed': len(ticktick_projects) if isinstance(ticktick_projects, list) else None,
    'ticktick_tasks_upserted': ticktick_tasks_upserted,
}
print(summary)


## Scheduling

You can schedule this notebook using any runner that can execute notebooks. Two common options:

- papermill (recommended):
  - Install: `pip install papermill`
  - Cron example: `papermill notebooks/scheduled_sync.ipynb /tmp/scheduled_sync_out.ipynb`
- jupyter nbconvert (execute):
  - Install: `pip install jupyter`
  - Cron example: `jupyter nbconvert --to notebook --execute notebooks/scheduled_sync.ipynb --output /tmp/scheduled_sync_out.ipynb`

Set environment variables for parameters as needed (e.g. in crontab or systemd unit):

- `DATABASE_URL` (required)
- `STRAVA_CLIENT_ID`, `STRAVA_CLIENT_SECRET`, `STRAVA_REFRESH_TOKEN` (or use stored token)
- `TICKTICK_CLIENT_ID`, `TICKTICK_CLIENT_SECRET`, `TICKTICK_REFRESH_TOKEN` (or use stored token)
- Optional: `STRAVA_PER_PAGE`, `STRAVA_PAGE_START`, `STRAVA_AFTER_EPOCH`, `TICKTICK_PROJECT_IDS`

If running from the repo root, `.env` is loaded automatically.